# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
URL = 'https://habr.com/ru/all/'
res = requests.get(URL)

In [4]:
KEYWORDS = ['python', 'парсинг']

In [5]:
soup = BeautifulSoup(res.text, 'html.parser')

In [6]:
posts = soup.find_all('article', class_='post post_preview')

In [7]:
def get_url_data(url, page):
    if page != 1:
        url = url + '/page' + str(page)
    res = requests.get(url)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post post_preview')
    for post in posts:
        hubs = post.find_all('a', class_='post__title_link')
        post_time = post.find('span', class_='post__time').text
        for hub in hubs:
            hub_lower = hub.text.lower()
            for keyword in KEYWORDS:
                if keyword in hub_lower:
                    title_element = post.find('a', class_='post__title_link')
                    return post_time, title_element.text, title_element.attrs.get('href')

In [10]:
data_dict = {
    'date': [],
    'title': [],
    'link': [],
}

In [11]:
for page in range(1,10):
    if get_url_data(URL, page):
        print(f'Нашлось на станице новостей №{page}')
        date, title, link = get_url_data(URL, page)
        data_dict['date'].append(date)
        data_dict['title'].append(title)
        data_dict['link'].append(link)

Нашлось на станице новостей №3
Нашлось на станице новостей №4
Нашлось на станице новостей №5
Нашлось на станице новостей №8
Нашлось на станице новостей №9


In [12]:
filtered_news = pd.DataFrame(data_dict)
filtered_news

,date,title,link
0,вчера в 22:37,Многопоточное скачивание файлов с ftp python-с...,https://habr.com/ru/post/537774/
1,вчера в 18:27,Прокачиваем скрипты симуляции HDL с помощью Py...,https://habr.com/ru/post/537704/
2,16 января 2021 в 17:22,Как определять собственные классы исключений в...,https://habr.com/ru/company/piter/blog/537642/
3,15 января 2021 в 12:00,Используйте парсинг вместо контроля типов,https://habr.com/ru/company/vdsina/blog/537398/
4,14 января 2021 в 19:25,Голосовой ассистент на Python (Виталий alfa 2.0),https://habr.com/ru/post/537390/


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [13]:
import json

In [14]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [15]:
headers = {
    'Host': 'identityprotection.avast.com',
    'Connection': 'keep-alive',
    'Content-Length': '34',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'Accept': 'application/json, text/plain, */*',
    'Vaar-Version': '0',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://www.avast.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.avast.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
}

In [16]:
params = {
        'emailAddresses': ["yy@ya.ru","xx@ya.ru", "12@wwwww.ru"]
    }

In [17]:
res = requests.post(URL, headers=headers, json=params)
res

<Response [200]>

In [18]:
breaches = json.loads(res.text)

In [19]:
breaches['data']

{'fc-zenit.ru': {'yy@ya.ru': [{'breachId': 3701,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'cfire.mail.ru': {'yy@ya.ru': [{'breachId': 3164,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'storelp.ru': {'xx@ya.ru': [{'breachId': 14357,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'vk.com': {'yy@ya.ru': [{'breachId': 12,
    'usernameBreached': True,
    'passwordBreached': True}],
  'xx@ya.ru': [{'breachId': 12,
    'usernameBreached': True,
    'passwordBreached': True}]}}

In [20]:
email_leak_dict = {
    'email': [],
    'leak_date': [],
    'leak_source': [],
    'leak_title': [],
}

In [21]:
for key, value in breaches['summary'].items():
    print(key, value)

yy@ya.ru {'breaches': [12, 3164, 3701]}
xx@ya.ru {'breaches': [14357, 12]}
12@wwwww.ru {'breaches': []}


In [22]:
for key, value in breaches['summary'].items():
    email = key
    if not value['breaches']:
        email_leak_dict['email'].append(email)
        email_leak_dict['leak_date'].append('no breach')
        email_leak_dict['leak_source'].append('no breach')
        email_leak_dict['leak_title'].append('no breach')
    for breach in value['breaches']:
        leak_date = breaches['breaches'][str(breach)]['publishDate']
        leak_source = breaches['breaches'][str(breach)]['site']
        leak_title = breaches['breaches'][str(breach)]['description']
        email_leak_dict['email'].append(email)
        email_leak_dict['leak_date'].append(leak_date)
        email_leak_dict['leak_source'].append(leak_source)
        email_leak_dict['leak_title'].append(leak_title)

In [23]:
email_leak_dict

{'email': ['yy@ya.ru',
  'yy@ya.ru',
  'yy@ya.ru',
  'xx@ya.ru',
  'xx@ya.ru',
  '12@wwwww.ru'],
 'leak_date': ['2016-10-29T00:00:00Z',
  '2017-02-14T00:00:00Z',
  '2017-03-31T00:00:00Z',
  '2018-06-19T00:00:00Z',
  '2016-10-29T00:00:00Z',
  'no breach'],
 'leak_source': ['vk.com',
  'cfire.mail.ru',
  'fc-zenit.ru',
  'storelp.ru',
  'vk.com',
  'no breach'],
 'leak_title': ["Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. ",
  "In July and August of 2016, two criminals carried out attacks on three separate forums hosted by Mail.ru, including CFire. The hackers used known SQL injection vulnerabilities found in older vBulletin forum software to obtain access to 

In [24]:
email_leak = pd.DataFrame(email_leak_dict)
email_leak

,email,leak_date,leak_source,leak_title
0,yy@ya.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
1,yy@ya.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
2,yy@ya.ru,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
3,xx@ya.ru,2018-06-19T00:00:00Z,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
4,xx@ya.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
5,12@wwwww.ru,no breach,no breach,no breach
